# Assignment

We saw a few examples of machine learning algorithms, so at this point it seems like once you get the data ready, machine learning is just "plug and play", but not so fast! 

In this assignment, we learn about **regularization**! Regularization is a means by which we can control how much a machine learning algorithm learns. The controlling is done using a parameter that the algorithm, called the **shrinkage parameter**, which is just an argument of the algorithm. The shrinkage parameter is an example of a **hyper-parameter**. A hyper-parameter is like a knob: by specifying a different value for the hyper-parameter, we control how the training happens, a process called **hyper-parameter tuning**. We talk about this is more detail in the next lesson.

We will look at two examples of regularizaiton: LASSO and Ridge regression. Both LASSO and Ridge regression are implementations of linear regression where we try to minimize prediction error plus some penalty that depends on the model's parameters (or coefficients) and the shrinkage constant (`alpha` in the code below). LASSO penalizes the model's parameters using the sum of the **absolute values** of the parameters (this is also called **L1-regularization**), while Ridge does so based on the sum of the **squared values** of the parameters (this is also called **L2-regularization**). For reasons we cannot elaborate on here, **LASSO has the by-product that it also does feature selection**, whereas Ridge doesn't. So not all regularization results in feature selection.

In [ ]:
import numpy as np
import pandas as pd
from plotly import express as px
import sklearn, sklearn.metrics, sklearn.model_selection, sklearn.preprocessing, sklearn.linear_model
pd.options.plotting.backend = 'plotly'

## Root Mean Squared Error (RMSE)
Before we dive into regularization we need to have a metric that tells us if our regression is good or bad.  For regressions we measure success differently than with classifications.  Instead of accuracy, the primary measure is **Root Mean Squared Error** (**RMSE**).  A **RMSE** of zero occurs when the predictions are perfect. **RMSE** increases as the predictions get worse. Let's disect the term **Root Mean Squared Error** by understanding each word from right to left.

### Error
The **E** in **RMSE** stands for **E**rror ($e_i$).  The error ($e_i$) is the difference between the known test target value ($y_i$) of sample $i$ and the predicted target value ($\hat{y}_i$) based on the test inputs of sample $i$.
$$e_i = y_i - \hat{y}_i$$
where
- $e_i$ is the error of the $i$th sample
- $y_i$ is the test target value of the $i$th sample
- $\hat{y}_i$ is the predicted target value from the inputs of the $i$th sample

### Squared
The **S** in **RMSE** stands for squared.  The errors ($e_i$) are squared: $(e_i)^2$.  

### Mean
The **M** in **RMSE** stands for mean.  We get the mean squared error (**MSE**) by taking the mean of all the squared errors in the sample:
$$\text{MSE} = \frac{1}{n}\sum_{i}(e_i)^2$$
where
- $\text{MSE}$ is the mean squared error
- $n$ is the number of samples

### Root
The **R** in **RMSE** stands for root.  We get the root mean squared error (**RMSE**) by taking the square root mean of **MSE**
$$\text{RMSE} = \sqrt{\text{MSE}}$$

Both **MSE** and **RMSE** are used to measure the prediction error.  **RMSE** is often preferred because it is in the units of the target variable ($y$), whereas **MSE** is in units of the target variable squared ($y^2$).

---

### Coding starts here
0. Compute error metrics.  
<br/>
To make sure we understand RMSE, lets do one small exercise.

We have the following data:
- The actual test target values are: `20, 15, 12, 17, 15, 14, 19, 17`
- The predicted target values predicted from test inputs are: `19, 12, 12, 19, 17, 17, 17, 18`

Determine and show
1. the 8 errors
2. the 8 squared errors derived from the errors
3. the MSE derived from the 8 squared errors
4. the RMSE derived from the MSE
5. the RMSE derived from the actual test target values and the predicted target values using sklearn.metrics.mean_squared_error with squared=False
<br/><span style="color:red" float:right>[1 point]</span>

In [ ]:
# Add Code here
err_dat = pd.DataFrame([20, 15, 12, 17, 15, 14, 19, 17], columns=['values'])
err_dat['predicted'] = [19, 12, 12, 19, 17, 17, 17, 18]
err_dat['error'] = err_dat['values'] - err_dat['predicted']
err_dat['squares'] = err_dat['error'] ** 2

err_dat

In [ ]:
mse = err_dat['squares'].mean()
rmse = np.sqrt(mse)
sk_rmse = sklearn.metrics.mean_squared_error(err_dat['values'],err_dat['predicted'], squared=False)

print(f"The MSE is {mse}\nThe RMSE is {rmse}\nAnd SciKit RMSE {sk_rmse}")

## Regularization
Let's get back to the main portion of this assignment.  As always we will need data before we can do our work.  Here we will use the California Housing Data from the 1990 census.  We will create regressions that predict the median house value in Californian communities.  You can find a description here:  https://www.kaggle.com/datasets/camnugent/california-housing-prices

In [ ]:
# Get California Housing Data
from sklearn.datasets import fetch_california_housing
HousingCal = fetch_california_housing()
# 20640 rows × 9 columns
# MedInc, HouseAge, AveRooms, AveBedrms, Population, AveOccup, Latitude, Longitude, MedHouseVal
df_housing = pd.DataFrame(data=HousingCal['data'], columns=HousingCal['feature_names'])
df_housing[HousingCal['target_names'][0]] = HousingCal['target']

Present the size of the data frame and the first few rows and a histogram for `MedHouseVal`. 

<span style="color:red" float:right>[0 point]</span>

In [ ]:
# Add code here
display(df_housing.shape,df_housing['MedHouseVal'])

In [ ]:
px.histogram(df_housing['MedHouseVal'],template='seaborn')

It is better to create a model that is limited in its scope, than to include data that are difficult to interpret.  In this case, we opt for a model that will only determine median house values up to 5 ($500,000).  The median house values for communities greater or equal to 5, are mostly incorrect.  

1. Prepare the data by doing the following:

    1. remove the communities (rows) where `MedHouseVal` is greater or equal to 5 and present another histogram of `MedHouseVal`
    2. remove `Latitude` and `Longitude` because we haven't learned yet how to make these useful
    3. split the data into features and target
    4. split the features and target into training and test sets
    5. normalize the training and test features based on the training data

    <span style="color:red" float:right>[1 point]</span>

In [ ]:
# Add code here
# remove the houses where `MedHouseVal` is greater or equal to 5
df_housing.drop(df_housing[df_housing['MedHouseVal'] >= 5].index,inplace=True)
# Present histogram of MedHouseVal
px.histogram(df_housing['MedHouseVal'],template='seaborn')

In [ ]:
# Add code here
# remove `Latitude` and `Longitude`
df_housing.drop(labels=['Latitude','Longitude'], axis=1, inplace=True)
df_housing.describe()

In [ ]:
# Add code here
# split the data into features and target
features = df_housing.drop('MedHouseVal',axis=1)
target = df_housing['MedHouseVal']
# split the features and target into training and test sets
feat_train, feat_test, targ_train, targ_test = sklearn.model_selection.train_test_split(features,target,test_size=0.15,random_state=41)


In [ ]:
feat_test.sort_index()

In [ ]:
# normalize the training and test features based on the training data
normie = sklearn.preprocessing.StandardScaler()
normie.fit(feat_train)
X_train = pd.DataFrame(normie.transform(feat_train),columns=feat_train.columns)
X_test = pd.DataFrame(normie.transform(feat_test),columns=feat_test.columns)

X_train

2. Train and Predict
    - Train a linear regression algorithm, using `LinearRegression` from `sklearn.linear_model`, to predict `MedHouseVal` on the prepared data.
    - predict the median house prices for both training and test data sets.  These predictions will serve as benchmarks.
    
    <span style="color:red" float:right>[1 point]</span>

In [ ]:
# Add code here
y_train = targ_train
linreg = sklearn.linear_model.LinearRegression()
linreg.fit(X_train, y_train)

y_hat_train_benchmark = linreg.predict(X_train)
y_hat_test_benchmark = linreg.predict(X_test)

3. Evaluate the model

    Calculate and report the performance (RMSE) on both training and test data. These numbers will serve as our benchmark performance. As a sanity check, create a scatter plot of the bench mark predictions against the actual values. Create this plot for both the training and test data sets.

    <span style="color:red" float:right>[2 point]</span>

In [ ]:
# Add code here
# Calculate and report RMSE
print(f"""The training benchmark is: {sklearn.metrics.mean_squared_error(targ_train.values,y_hat_train_benchmark, squared=False):0.5f}
The testing benchmark is: {sklearn.metrics.mean_squared_error(targ_test.values,y_hat_test_benchmark, squared=False):0.5f}""")

In [ ]:
# Add code here
evaluation = pd.DataFrame(targ_test.values,columns=['target_values'])
evaluation['linear_predicted_values'] = y_hat_test_benchmark
# scatter plot of the bench mark predictions against the actual values
px.scatter(evaluation,x='target_values',y='linear_predicted_values',template='seaborn')

We now train a regularized version of `LinearRegression` called `Lasso` (you can load it from the sklearn.linear_model.Lasso). `Lasso` has an argument called `alpha`, which is the **shrinkage parameter** we referred to earlier.

4. Let `alpha = 0.000001` and train a `Lasso` model. Show that the resulting model is practically identical to the one we trained with `LinearRegression`. There are different ways to show this, so you will need to think of a way. 

    <span style="color:red" float:right>[2 point]</span>

In [ ]:
lasso = sklearn.linear_model.Lasso(alpha=0.000001)
lasso.fit(X_train,y_train)

In [ ]:
evaluation['lasso_predicted_values'] = lasso.predict(X_test)
px.histogram(
    evaluation.diff(axis=1)['lasso_predicted_values'],
    x='lasso_predicted_values',
    template='seaborn',
    title='Difference between Linear and Lasso (alpha=0.000001)',
    labels={'lasso_predicted_values':'difference'}
)

In [ ]:
fig = px.scatter(evaluation,x='linear_predicted_values',y='lasso_predicted_values')
fig.add_shape(type='line',x0=-3,y0=-3,x1=9,y1=9, line=dict(color="LightSeaGreen",width=1,dash="dot"))

#### Comment on the comparison

> The difference between the Linear regression and the Lasso regression with alpha at 0.000001 is so small it is insignificant. Most values are with 0.000004

5. Get regression coefficients and performance metrics for varying alpha.  The performance metrics are  RMSE for training and test data.  

    <span style="color:red" float:right>[5 point]</span>

In [ ]:
# The alpha column contains the values we want to iterate over
# The other columns are for storing the coefficients of the lasso regressions
X = feat_train
Coefficients = pd.DataFrame(data=np.arange(0.01, 1, .01), columns=['alpha'])
Coefficients[list(X.columns) + ['RMSE_train', 'RMSE_test']] = 0.

To make it easier, we have laid out a suggestion for the code. You can use our layout or you can design your own code.
For each iteration in the suggested design:    
- Train a new `Lasso` model, letting `alpha` change each time to one of the values given by `alpha_vals` below.  
- For each value of `alpha` in `Coefficients`, retrieve the coefficients from the `coef_` attribute of the trained model. 
- Store the coefficients in `Coefficients`
- Predict the training labels, e.g. `y_hat_train`
- Determine the rmse of the training data, e.g. `rmse_train`
- Store the rmse of the training data in `Coefficients`
- Predict the test labels, e.g. `y_hat_test`
- Determine the rmse of the test data, e.g. `rmse_test`
- Store the rmse of the test data in `Coefficients`

In [ ]:
Coefficients

In [ ]:
# Add code here
# loop over the alpha values and do this:
for n,alpha in enumerate(Coefficients.alpha):
    # instantiate a Lasso regression with alpha
    lasso_run = sklearn.linear_model.Lasso(alpha=alpha)
    # fit the lasso regression
    lasso_run.fit(X_train,y_train)
    # add the coefficients to the DataFrame row
    Coefficients.loc[n,X.columns] = lasso_run.coef_
    # predict training labels (y_hat for training)
    # determine training error metric (RMSE)
    # add the training error metric (RMSE) to the DataFrame row
    Coefficients.loc[n,'RMSE_train'] = sklearn.metrics.mean_squared_error(targ_train.values,lasso_run.predict(X_train), squared=False)
    # predict test labels (y_hat for test)
    # determine test error metric (RMSE)
    # add the test error metric (RMSE) to the DataFrame row
    Coefficients.loc[n,'RMSE_test'] = sklearn.metrics.mean_squared_error(targ_test.values,lasso_run.predict(X_test), squared=False)



In [ ]:
Coefficients.head(20)

6. Lasso changes with alpha

    (a) Using a visual, show how the performance on the training and test data changed as we gradually increased the shrinkage parameter `alpha`. At which point is there a substantial decrease in performance (increase in RMSE) on the test data?  You might want to use a log scale (`plt.xscale('log')`) for `alpha`.

    (b) Using a visual, show how the model's coefficients changed as we gradually increased the shrinkage parameter `alpha`. HINT: They should appear to be shrinking toward zero as you increase `alpha`. 

    <span style="color:red" float:right>[4 point]</span>

In [ ]:
# Add code here
# RMSE vs. alpha
px.bar(Coefficients[:70],x='alpha',y=['RMSE_train','RMSE_test'],template='seaborn',barmode='group')

In [ ]:
# Add code here
# Coefficient vs. alpha
px.bar(Coefficients[:70],x='alpha',y=list(X.columns))

7. Compare `Ridge` and `Lasso` baseline models.  Show that `Ridge` and `Lasso` return the same trained model when `alpha = 0.00001` (i.e. close to zero). <br/><span style="color:red" float:right>[1 point]</span>

In [ ]:
# Add code here
ridge = sklearn.linear_model.Ridge(alpha=0.000001)
# fit the ridge regression
ridge.fit(X_train,y_train)
# compare the coefficients
print(f"""Ridge coefficients: {ridge.coef_}
Lasso coefficients: {lasso.coef_}\n""")
# compare predicted test labels (y_hat for test)
print(f"""Ridge predictions: {ridge.predict(X_test)[:6]}
Lasso predictions: {lasso.predict(X_test)[:6]}""")

8. Get regression coefficients and performance metrics for varying `alpha` on a ridge regression.  The performance metrics are  RMSE for training and test data.  In other words, repeat the above using `Ridge` instead of `Lasso`.   For `ridge`, we will use the following range for `alpha`: `np.power(2., np.arange(1., 25., 1.))`

    > Note: Ridge has a  different behaviour than Lasso. Coefficients will very gradually approach zero and not have a steep drop off. You probably will not notice any zero coefficients.

    <span style="color:red" float:right>[1 point]</span>

In [ ]:
# Add code here
# Determine alpha_values
# The alpha column contains the values we want to iterate over
# The other columns are for storing the coefficients of the ridge regressions
Coefficients_R = pd.DataFrame(data=np.power(2., np.arange(1., 25., 1.)), columns=['alpha'])
Coefficients_R[list(X.columns) + ['RMSE_train', 'RMSE_test']] = np.nan

for n,alpha in enumerate(Coefficients_R.alpha):
    lasso_run = sklearn.linear_model.Ridge(alpha=alpha)
    lasso_run.fit(X_train,y_train)
    Coefficients_R.loc[n,X.columns] = lasso_run.coef_
    Coefficients_R.loc[n,'RMSE_train'] = sklearn.metrics.mean_squared_error(targ_train.values,lasso_run.predict(X_train), squared=False)
    Coefficients_R.loc[n,'RMSE_test'] = sklearn.metrics.mean_squared_error(targ_test.values,lasso_run.predict(X_test), squared=False)

In [ ]:
Coefficients_R.head(20)

9. Ridge changes with alpha
<br/>
(a) Using a visual, show how the performance on the training and test data changed as we gradually increased `alpha` for `Ridge`. In other words, repeat the above using `Ridge` instead of `Lasso`.  
<br/>
(b) Using a visual, show how the model's coefficients for ridge changed as we gradually increased the shrinkage parameter `alpha`.  
<br/><span style="color:red" float:right>[2 point]</span>

In [ ]:
# Add code here
# RMSE vs. alpha
px.scatter(Coefficients_R,x='alpha',y=['RMSE_train','RMSE_test'],template='seaborn',log_x=True)

In [ ]:
# Add code here
# Coefficient vs. alpha
px.scatter(Coefficients_R[:25],x='alpha',y=list(X.columns),log_x=True)

10. Based on the results, briefly describe the effect of changing `alpha` on the coefficients of both `Ridge` and `Lasso`. What value of `alpha` would you choose for each case? You do not need to give a precise answer, but choose a number. <br/><span style="color:red" float:right>[2 point]</span>

#### Comments on regularization using ridge and lasso

> Changing the `alpha` of either `Ridge` or `Lasso` seems to decrease the effect of all coefficients. In both it seems to leave an isolated single variable as the predictor, at least for this dataset. 

> For the `Lasso` regression I would choose a number around `0.005 to 0.01` and for the `Ridge` regression I would choose something between `1 to 5`.

# End of assignment